# Agent CFO — Performance Optimization & Design

---
This is Group 8's notebook for the ICT3113 Performance Testing and Optimization project following the required structure.


You will design and optimize an Agent CFO assistant for a listed company. The assistant should answer finance/operations questions using RAG (Retrieval-Augmented Generation) + agentic reasoning, with response time (latency) as the primary metric.

Your system must:
*   Ingest the company’s public filings.
*   Retrieve relevant passages efficiently.
*   Compute ratios/trends via tool calls (calculator, table parsing).
*   Produce answers with valid citations to the correct page/table.


## 1. Config & Secrets

Fill in your API keys in secrets. **Do not hardcode keys** in cells.

In [22]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

COMPANY_NAME = "NVIDIA"

## 2. Data Download (Dropbox)

*   Annual Reports: last 3–5 years.
*   Quarterly Results Packs & MD&A (Management Discussion & Analysis).
*   Investor Presentations and Press Releases.
*   These files must be submitted later as a deliverable in the Dropbox data pack.
*   Upload them under `/content/data/`.

Scope limit: each team will ingest minimally 15 PDF files total.


## 3. System Requirements

**Retrieval & RAG**
*   Use a vector index (e.g., FAISS, LlamaIndex) + a keyword filter (BM25/ElasticSearch).
*   Citations must include: report name, year, page number, section/table.

**Agentic Reasoning**
*   Support at least 3 tool types: calculator, table extraction, multi-document compare.
*   Reasoning must follow a plan-then-act pattern (not a single unstructured call).

**Instrumentation**
*   Log timings for: T_ingest, T_retrieve, T_rerank, T_reason, T_generate, T_total.
*   Log: tokens used, cache hits, tools invoked.
*   Record p50/p95 latencies.

In [23]:
# Install dependencies for ingestion pipeline
%pip install langchain-community langchain-openai faiss-cpu rank-bm25 openai tiktoken pypdf pdfplumber

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
%pip install pyinstrument

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Import libraries for ingestion pipeline

In [25]:
import time
import glob
import json
import re
import pdfplumber
from collections import defaultdict
from concurrent.futures import ProcessPoolExecutor, ThreadPoolExecutor
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.retrievers import BM25Retriever
from langchain.schema import Document
from langchain.retrievers import SVMRetriever
from langchain_core.retrievers import BaseRetriever
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.tools import StructuredTool
from langchain.agents import initialize_agent, AgentType
from typing import List, Dict, Optional

PDF Parser and Chunking (Using pdfplumber)
1. Extracts words from pdf
2. Detects sections
3. Returns langchain Document object with page content and metadata containing page number, report name, section, and type of content

In [26]:
def detect_section(page, text, min_size_diff=2.0):
    """
    Try to extract the section title from page text using regex patterns or font size, weight.
    Handles 10-K, 10Q, Presentations, Investor PDFs and similar formats.
    """
    patterns = [
        r"(Item\s+\d+[A-Za-z]?.\s*[A-Z][^\n]+)",  # e.g. "Item 7. Management’s Discussion..."
        r"(ITEM\s+\d+[A-Z]?.\s*[A-Z][^\n]+)",     # uppercase variant
        r"(^[A-Z][A-Z\s]{10,})"                   # fallback for all-caps section titles
    ]
    for p in patterns:
        m = re.search(p, text)
        if m:
            return m.group(1).strip()

    # For presentation decks/investor pdfs
    try:
        words = page.extract_words(extra_attrs=["size", "fontname"])
        if not words:
            return None

        from collections import defaultdict
        lines = defaultdict(list)

        # Group words by y-position
        for w in words:
            lines[round(w["top"], -1)].append(w)

        # Compute global average font size to compare against
        avg_font_size = sum(float(w["size"]) for w in words) / len(words)

        # Define line scoring function
        def score_line(line):
            avg_size = sum(float(w["size"]) for w in line) / len(line)
            bold_bonus = any("Bold" in w["fontname"] or "Black" in w["fontname"] for w in line)
            return avg_size + (3 if bold_bonus else 0), avg_size

        # Score all lines
        scored_lines = [(y, line, *score_line(line)) for y, line in lines.items()]
        if not scored_lines:
            return None

        # Pick the top scoring line
        _, best_line, best_score, best_size = max(scored_lines, key=lambda x: x[2])

        # If the line isn't significantly larger or bold, discard it
        if best_size < avg_font_size + min_size_diff:
            # (e.g. all text is 10pt, best line is 11pt → not a section)
            return None

        # Skip lines that look too long (to avoid full sentences)
        if len(best_line) > 20:
            return None

        section_title = " ".join(w["text"] for w in best_line)
        return section_title.strip() if section_title else None

    except Exception:
        return None

    return None


def parse_pdf_with_tables(pdf_path, report_name, year=None):
    text_docs, table_docs = [], []
    current_section = None  # rolling context

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages, start=1):
            text = page.extract_text() or ""
            tables = page.extract_tables() or []

            # --- Try to detect section header ---
            section = detect_section(page, text)
            if section:
                current_section = section  # update rolling section

            # --- Create text Document ---
            text_docs.append(
                Document(
                    page_content=text,
                    metadata={
                        "page": i,
                        "report": report_name,
                        "year": year,
                        "section": current_section,
                        "type": "text"
                    }
                )
            )

            # --- Create table Documents ---
            for t in tables:
                cleaned_table = [
                    [cell if cell is not None else "" for cell in row]
                    for row in t
                ]
                table_text = "\n".join(["\t".join(row) for row in cleaned_table])
                table_docs.append(
                    Document(
                        page_content=table_text,
                        metadata={
                            "page": i,
                            "report": report_name,
                            "year": year,
                            "section": current_section,
                            "type": "table"
                        }
                    )
                )

    return text_docs, table_docs

Main ingestion pipeline

1.   Loops over all pdfs in folder
2.   Checks cache for parsed pdf
3.   Load from cache if exists, avoiding re-parsing pdf
4.   Save to cache after parsing
5.   Use parallel processing to parse pdfs in parallel



In [27]:
# --- Parameters ---
pdf_folder = "data/"

PARSED_DIR = "cache/parsed_pdfs"
os.makedirs(PARSED_DIR, exist_ok=True)

def parse_pdf_cached(pdf_path, report_name, year=None):
    cache_file = os.path.join(PARSED_DIR, f"{report_name}.json")

    try:
        # --- If cached and file not modified, skip re-parsing ---
        pdf_mtime = os.path.getmtime(pdf_path)
        if os.path.exists(cache_file):
            # cache_mtime = os.path.getmtime(cache_file)
            # if cache_mtime > pdf_mtime:
            print(f"Using cached parse for {report_name}\n")
            with open(cache_file, "r", encoding="utf-8") as f:
                data = json.load(f)
            docs = [Document(**d) for d in data]
            return docs

        # --- Otherwise, parse fresh ---
        print(f"Parsing {os.path.basename(pdf_path)} ...\n")
        text_docs, table_docs = parse_pdf_with_tables(pdf_path, report_name, year)
        docs = text_docs + table_docs

        # Save to cache
        serializable_docs = [
            {"page_content": d.page_content, "metadata": d.metadata}
            for d in docs
        ]
        with open(cache_file, "w", encoding="utf-8") as f:
            json.dump(serializable_docs, f, ensure_ascii=False, indent=2)

        return docs
    except Exception as e:
        print(f"Error parsing PDF {report_name}: {e}")
        return []


# --- Measure time!! ----
start_total = time.perf_counter()
all_docs = []

def process_pdf(pdf_path):
    report_name = os.path.basename(pdf_path).replace(".pdf", "")
    return parse_pdf_cached(pdf_path, report_name)

for pdf_path in glob.glob(os.path.join(pdf_folder, "*.pdf")):
    doc = process_pdf(pdf_path)
    all_docs.extend(doc)

# with ThreadPoolExecutor(max_workers=4) as executor:
#     results = list(executor.map(process_pdf, glob.glob(os.path.join(pdf_folder, "*.pdf"))))
#     for docs in results:
#         all_docs.extend(docs)

elapsed_total = time.perf_counter() - start_total
print(f"Loaded {len(all_docs)} parsed documents in {elapsed_total:.2f}s.")


Using cached parse for FY21_10K

Using cached parse for FY22_10K

Using cached parse for FY23_10K

Using cached parse for FY24Q1_10Q

Using cached parse for FY24Q2_10Q

Using cached parse for FY24Q3_10Q

Using cached parse for FY24_10K

Using cached parse for FY25Q1_10Q

Using cached parse for FY25Q2_10Q

Using cached parse for FY25Q2_QuarterlyPresentation

Using cached parse for FY25Q3_10Q

Using cached parse for FY25Q3_QuarterlyPresentation

Using cached parse for FY25Q4_PR

Using cached parse for FY25Q4_QuarterlyPresentation

Using cached parse for FY25_10K

Using cached parse for FY26Q1_10Q

Using cached parse for FY26Q1_PR

Using cached parse for FY26Q1_QuarterlyPresentation

Using cached parse for FY26Q2_10Q

Using cached parse for FY26Q2_PR

Using cached parse for FY26Q2_QuarterlyPresentation

Loaded 2039 parsed documents in 0.57s.


Chunking
1. Uses langchain's Recursive Character Text splitter to split documents
2. Returns chunked documents

In [28]:
from pyinstrument import Profiler
prof = Profiler(async_mode="disabled")
prof.start()

start_total = time.perf_counter()

max_chunk_size = 800
chunk_overlap = 100

def chunk_documents(all_docs, chunk_size=max_chunk_size, chunk_overlap=chunk_overlap):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return splitter.split_documents(all_docs)

chunked_docs = chunk_documents(all_docs, max_chunk_size, chunk_overlap)
print(f"Chunked into {len(chunked_docs)} total segments.")

print(f"Parsed and chunked {len(all_docs)} total text segments from {len(glob.glob(pdf_folder + '*.pdf'))} PDFs.")
elapsed_total = time.perf_counter() - start_total

prof.stop()
print(prof.output_text(unicode=True, color=True))
prof.last_session.save("01_Chunking.pyisession")
html_output = prof.output_html()
with open("01_Chunking.html", "w", encoding="utf-8") as f:
    f.write(html_output)
print(f"Total elapsed time: {elapsed_total}")

Chunked into 6354 total segments.
Parsed and chunked 2039 total text segments from 21 PDFs.

  _     ._   __/__   _ _  _  _ _/_   Recorded: 02:21:09  Samples:  356
 /_//_/// /_\ / //_// / //_'/ //     Duration: 0.791     CPU time: 0.766
/   _/                      v5.1.1

Profile at C:\Users\cryst\AppData\Local\Temp\ipykernel_35640\492970387.py:3

0.790 <module>  ..\..\..\Temp\ipykernel_35640\492970387.py:1
├─ 0.780 chunk_documents  ..\..\..\Temp\ipykernel_35640\492970387.py:10
│  └─ 0.780 RecursiveCharacterTextSplitter.split_documents  langchain_text_splitters\base.py:97
│        [24 frames hidden]  langchain_text_splitters, langchain_c...
│           0.458 Document.__init__  langchain_core\load\serializable.py:113
│           └─ 0.445 [self]  langchain_core\load\serializable.py
└─ 0.009 [self]  ..\..\..\Temp\ipykernel_35640\492970387.py


Total elapsed time: 0.7895931000020937


## 4. Baseline Pipeline

**Baseline (starting point)**
*   Naive chunking.
*   Single-pass vector search.
*   One LLM call, no caching.

In [29]:
# # TODO: Implement baseline retrieval + generation
# # Take a query -> Retrieve relevant chunks -> Feed to LLM -> answer

# # number of docs to retrieve per query
# bm25_k = 10
# faiss_k = 12

# # FAISS Vector Index
# faiss_store = FAISS.from_documents(chunked_docs, OpenAIEmbeddings(model="text-embedding-3-small"))
# faiss_store.save_local("faiss_index")

# print("FAISS vector index built and saved locally.")

# # BM25 Keyword Index
# bm25_retriever = BM25Retriever.from_documents(chunked_docs)
# bm25_retriever.k = bm25_k

# print("BM25 keyword retriever ready.")

# print("Ingestion complete — FAISS and BM25 indices are ready.\n")

# # Test Prompt
# question = "Show Operating Expenses for the last 3 fiscal years, year-on-year comparison."

# # Test FAISS vector index
# faiss_retriever = faiss_store.as_retriever(search_kwargs={"k": faiss_k})
# test_faiss_docs = faiss_retriever.invoke(question)
# print('Using FAISS')
# print(test_faiss_docs[0].page_content)
# print(test_faiss_docs[0].metadata)
# print('\n')

# # Test BM25 retriever
# test_bm25_docs = bm25_retriever.invoke(question)
# print('Using BM25')
# print(test_bm25_docs[0].page_content)
# print(test_bm25_docs[0].metadata)
# print('\n')

# # Test SVM retriever
# # svm_retriever = SVMRetriever.from_documents(chunked_docs, embeddings=embedding_model)
# # svm_retriever.k = 10
# # test_svm_docs = svm_retriever.invoke(question)
# # print('Using SVM:')
# # print(test_svm_docs[0].page_content)
# # print(test_svm_docs[0].metadata)

# context = "\n\n".join([doc.page_content for doc in test_faiss_docs])

In [30]:
# for doc in test_faiss_docs:
#   print("============================")
#   print(doc.page_content)
#   print(doc.metadata)

In [31]:
# # Set Prompts
# from langchain_core.prompts import PromptTemplate

# prompt_template = \
# """
# You are a helpful assistant for question answering tasks.
# Use the following pieces of retrieved context to answer the given question.
# If you dont know the answer, just say that you dont know.
# Use up to three sentenses to keep answer precise.

# Question: {question}

# Context: {context}
# """

# prompt_template = PromptTemplate.from_template(prompt_template)
# # Invoke and pass in query and context
# prompt = prompt_template.invoke(
#     {
#         "context": context,
#         "question": question
#     }
# )
# prompt.text

In [32]:
# # Generate Response
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
# response = llm.invoke(prompt)

# # Parse the response into well formated text
# from langchain_core.output_parsers import StrOutputParser
# parser = StrOutputParser()

# res = parser.invoke(response)

# print(res)

## 4b. Agentic RAG

In [33]:
# Retriever Tool
class RetrieverTool:
    """Wrapper around FAISS retriever for LangChain/OpenAI tools interface."""

    def __init__(self, faiss_store: FAISS, top_k: int = 12):
        self.retriever: BaseRetriever = faiss_store.as_retriever(
            search_kwargs={"k": top_k}
        )

    def forward(self, query: str) -> str:
        """Retrieve top-k documents similar to the query."""
        assert isinstance(query, str), "Your search query must be a string"
        docs = self.retriever.invoke(query)

        if not docs:
            return "No relevant documents found."

        formatted_docs = "\n".join(
            [
                f"===== Document {i+1} =====\n"
                f"{doc.page_content}\n"
                f"Metadata: {doc.metadata}\n"
                for i, doc in enumerate(docs)
            ]
        )
        return formatted_docs

    def as_tool(self) -> StructuredTool:
        """Convert to a LangChain StructuredTool usable by OpenAI agents."""
        return StructuredTool.from_function(
            func=self.forward,
            name="retriever",
            description="Retrieves semantically similar documents from a FAISS vector index given a natural language query.",
        )

In [34]:
# Comparison Tools
class ComparisonTool:
    """Tool to compute Year-over-Year (YoY) or Quarter-over-Quarter (QoQ) percentage changes from numeric data."""

    @staticmethod
    def forward(input_str: str) -> str:
        # Clean and sanitize LLM formatting
        input_str = re.sub(r"^```(?:json)?|```$", "", input_str.strip(), flags=re.IGNORECASE).strip()

        try:
            params = json.loads(input_str)
        except Exception as e:
            return json.dumps({"error": f"Invalid input JSON: {str(e)}", "received": input_str})

        # --- Parse parameters ---
        data = params.get("data", [])
        metric_key = params.get("metric_key", "opex")
        period_key = params.get("period_key", "fiscal_year")
        comparison_type = params.get("comparison_type", "yoy")

        # --- Handle dict input ---
        if isinstance(data, dict):
            data = [{period_key: k, metric_key: v} for k, v in data.items()]

        if not isinstance(data, list):
            return json.dumps({"error": "Expected 'data' to be a list or dict of period:value pairs."})

        # --- Sort chronologically if possible ---
        try:
            data = sorted(data, key=lambda x: x[period_key])
        except Exception:
            pass

        # --- Compute percentage changes ---
        results = []
        for i, record in enumerate(data):
            if i == 0:
                pct_change = None
            else:
                prev_value = data[i - 1][metric_key]
                curr_value = record[metric_key]
                pct_change = None if prev_value == 0 else ((curr_value - prev_value) / prev_value) * 100

            results.append({
                period_key: record[period_key],
                metric_key: record[metric_key],
                f"{comparison_type}_change (%)": round(pct_change, 2) if pct_change is not None else None,
                "units": record.get("units", "millions USD"),
            })

        return json.dumps(results, indent=2)

    def as_tool(self) -> StructuredTool:
        return StructuredTool.from_function(
            func=self.forward,
            name="comparison_tool",
            description=(
                "Computes Year-over-Year (YoY) or Quarter-over-Quarter (QoQ) percentage changes "
                "for a given financial metric across time periods. Input must be valid JSON with 'data', "
                "'metric_key', 'period_key', and 'comparison_type'."
            ),
        )

In [35]:
# Formula registry for common ratios
FINANCIAL_FORMULAS = {
    "operating_efficiency_ratio": {
        "formula": "opex / operating_income",
        "description": "Measures how much operating expense is spent per unit of operating income."
    },
    "gross_margin": {
        "formula": "(gross_profit / revenue) * 100",
        "description": "Percentage of revenue remaining after cost of goods sold (COGS)."
    },
    "net_margin": {
        "formula": "(net_income / revenue) * 100",
        "description": "Percentage of revenue retained as net income after expenses."
    },
    "r&d_to_revenue": {
        "formula": "(r_and_d / revenue) * 100",
        "description": "R&D expenses as a percentage of revenue."
    },
}


def resolve_formula_from_query(query: str) -> str | None:
    """Tries to resolve a natural language query to a known formula."""
    q = query.lower()
    for key, entry in FINANCIAL_FORMULAS.items():
        if key.replace("_", " ") in q:
            return entry["formula"]
        if any(kw in q for kw in key.split("_")):
            return entry["formula"]
    return None


# --- Calculator Tool ---
class CalculatorTool:
    """Dynamic Calculator Tool for RAG agents to compute arbitrary financial metrics."""

    @staticmethod
    def forward(input_str: str) -> str:
        """
        Accepts JSON input like:
        {
          "query": "Calculate Operating Efficiency Ratio for the last 3 fiscal years",
          "data": [...],
          "period_key": "fiscal_year",
          "formula": "opex / operating_income"
        }
        Returns: JSON string of computed results.
        """
        # Clean formatting
        input_str = re.sub(r"^```(?:json)?|```$", "", input_str.strip(), flags=re.IGNORECASE).strip()

        try:
            params = json.loads(input_str)
        except Exception as e:
            return json.dumps({"error": f"Invalid input JSON: {str(e)}"})

        query = params.get("query", "").lower()
        data = params.get("data", [])
        period_key = params.get("period_key", "fiscal_year")
        formula = params.get("formula")

        if isinstance(data, dict):
            data = [{period_key: k, **v} for k, v in data.items()]

        if not isinstance(data, list):
            return json.dumps({"error": "Expected 'data' to be a list or dict of period:value mappings."})

        # If no formula is provided, attempt to infer it from the query
        if not formula:
            formula = resolve_formula_from_query(query)
        if not formula:
            return json.dumps({"error": "No formula found or inferred from query."})

        # Identify variables referenced in the formula
        variables = re.findall(r"[a-zA-Z_][a-zA-Z0-9_]*", formula)

        results = []
        for record in data:
            local_vars = {var: record.get(var) for var in variables}
            if None in local_vars.values():
                results.append({
                    period_key: record.get(period_key),
                    "error": f"Missing data for: {', '.join([k for k,v in local_vars.items() if v is None])}"
                })
                continue

            try:
                # Safe evaluation environment
                value = eval(formula, {"__builtins__": {}}, local_vars)
            except Exception as e:
                results.append({
                    period_key: record.get(period_key),
                    "error": f"Computation failed: {str(e)}"
                })
                continue

            results.append({
                period_key: record[period_key],
                "computed_value": round(value, 4),
                "formula_used": formula,
                "working": f"{formula.replace('/', ' ÷ ')} = {round(value, 4)}"
            })

        return json.dumps(results, indent=2)

    def as_tool(self):
        """Expose this as a LangChain-compatible tool."""
        return StructuredTool.from_function(
            func=self.forward,
            name="calculator_tool",
            description=(
                "Computes arbitrary financial ratios or derived metrics from structured data. "
                "Accepts a JSON input with fields 'query', 'data', 'period_key', and optionally 'formula'. "
                "Understands user queries like 'Calculate operating efficiency ratio' or custom formulas like "
                "'(gross_profit - opex) / revenue'."
            ),
        )

In [36]:
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small") # Can be 'text-embedding-3-large'

def build_faiss_index(chunked_docs, embedding_model):
  embedding_model = embedding_model

  # FAISS Vector Index
  faiss_store = FAISS.from_documents(chunked_docs, embedding_model)
  faiss_store.save_local("faiss_index")

  # Load FAISS Vector Store
  faiss_store = FAISS.load_local(
      "faiss_index",
      embeddings=embedding_model,
      allow_dangerous_deserialization=True
  )
  return faiss_store

In [76]:
def create_system_message(question):
  return f"""
You are a financial analyst agent that can use tools.

Use the retriever tool when you need to fetch financial data. You can only call this tool ONCE.
Use the comparison tool when you need to compute YoY or QoQ comparisons.
Use the calculator tool to compute or derive financial ratios or custom metrics (e.g., Opex ÷ Operating Income, Gross Margin %, Net Margin).

Once done, return
1. The **final structured JSON output** in this format:
2. **Prose explanation**, converting the JSON output into a formatted table

{{
"query": "...",
"data_values": [...],
"computed_values": [...],
"citations": [{{"report": "...", "page": ..., "section": "..."}}],
"tools": ["<list the tools you actually used>"],
"tools_count": <total number of tools used>
}}

Guidelines:
- `data_values` contain the raw financial figures, corresponding fiscal years, and units retrieved directly from reports before any calculations.
- `computed_values` include the calculated results (e.g., YoY or QoQ changes) together with the corresponding values from data_values.
- Always include every period in `computed_values`, even if the change value is null.

Now, handle this query:
{question}
"""

In [51]:
def create_agent(faiss_store):
  # Create Tools
  retriever_tool = RetrieverTool(faiss_store=faiss_store, top_k=12).as_tool()
  comparison_tool = ComparisonTool().as_tool()
  calculator_tool = CalculatorTool().as_tool()

  # Initialize OpenAI LLM
  openai_llm = ChatOpenAI(model="gpt-4o", temperature=0, streaming=True)

  # Create OpenAI-compatible agent that can use tools
  agent = initialize_agent(
      tools=[retriever_tool, comparison_tool, calculator_tool],
      llm=openai_llm,
      agent_type=AgentType.OPENAI_FUNCTIONS,  # enables OpenAI’s function/tool calling
      verbose=True,
      handle_parsing_errors=True
  )
  return agent

In [70]:
from langchain.callbacks.base import BaseCallbackHandler

# class TimingCallback(BaseCallbackHandler):
#     def __init__(self):
#         self.tool_timings = []
#         self.current_tool = None
#         self.start_time = None
#         self.llm_start = None
#         self.reasoning_time = None

#     def on_tool_start(self, serialized, input_str, **kwargs):
#         self.current_tool = serialized.get("name", "UnknownTool")
#         self.start_time = time.perf_counter()
#         print(f"Tool start: {self.current_tool}")

#     def on_tool_end(self, output, **kwargs):
#         duration = time.perf_counter() - self.start_time
#         print(f"Tool end: {self.current_tool} ({duration:.3f}s)")
#         self.tool_timings.append((self.current_tool, duration))
#         self.current_tool = None

#     def on_llm_start(self, *args, **kwargs):
#         self.llm_start = time.perf_counter()

#     def on_llm_end(self, response, **kwargs):
#         if self.llm_start:
#             duration = time.perf_counter() - self.llm_start
#             print(f"LLM reasoning/generation: {duration:.3f}s")
#             self.reasoning_time = duration

class TimingCallback(BaseCallbackHandler):
    """
    Tracks:
    - Tool timings (per tool)
    - Token usage per LLM call
    - Consolidated LLM reasoning time
    - Total session time
    """

    def __init__(self):
        self.tool_timings = []
        self.current_tool = None
        self.tool_start_time = None

        self.llm_start_time = None
        self.reasoning_total = 0.0
        self.generate_total = 0.0
        
        # NEW: Track when last tool ended
        self.last_tool_end_time = None
        self.generation_start_time = None

        # --- Token counts ---
        self.prompt_tokens = 0
        self.completion_tokens = 0
        self.total_tokens = 0
        
        # NEW: Track tokens per LLM call
        self.llm_calls = []
        self.llm_call_count = 0
        
        # NEW: Track context for current LLM call
        self.llm_context = None  # Will be set in on_llm_start

        self.session_start = time.perf_counter()
        self.session_end = None

    # -----------------------
    # TOOL TIMING
    # -----------------------
    def on_tool_start(self, serialized, input_str, **kwargs):
        self.current_tool = serialized.get("name", "UnknownTool")
        self.tool_start_time = time.perf_counter()
        print(f"[Tool start] {self.current_tool}")

    def on_tool_end(self, output, **kwargs):
        if self.tool_start_time:
            duration = time.perf_counter() - self.tool_start_time
            self.tool_timings.append((self.current_tool, duration))
            print(f"[Tool end] {self.current_tool} ({duration:.3f}s)")
            # Track when the last tool ended
            self.last_tool_end_time = time.perf_counter()
        self.current_tool = None
        self.tool_start_time = None

    # -----------------------
    # LLM REASONING TIMING
    # -----------------------
    def on_llm_start(self, *args, **kwargs):
        self.llm_start_time = time.perf_counter()
        self.llm_call_count += 1
        
        # Capture the context at START time
        if self.current_tool:
            self.llm_context = f"during_{self.current_tool}"
        else:
            # Determine purpose based on call number
            if self.llm_call_count == 1:
                self.llm_context = "initial_planning"
            else:
                self.llm_context = "reasoning"

    def on_llm_end(self, response, **kwargs):
        # Timing
        if self.llm_start_time:
            duration = time.perf_counter() - self.llm_start_time
            self.reasoning_total += duration
            print(f"[LLM reasoning segment] {duration:.3f}s")
        self.llm_start_time = None

        # Token usage extraction
        try:
            usage = None
            
            # Method 1: Check llm_output
            # if hasattr(response, 'llm_output') and response.llm_output:
                # usage = response.llm_output.get('token_usage', None)
            
            # Method 2: Check usage_metadata attribute directly
            # if not usage and hasattr(response, 'usage_metadata'):
            #     usage = response.usage_metadata
            
            # Method 3: Check generations -> message -> usage_metadata
            if not usage and hasattr(response, 'generations'):
                for gen_list in response.generations:
                    if isinstance(gen_list, list):
                        for gen in gen_list:
                            if hasattr(gen, 'message') and hasattr(gen.message, 'usage_metadata'):
                                usage = gen.message.usage_metadata
                                break
                    if usage:
                        break
            
            # Extract tokens if we found usage data
            if usage:
                # Handle both dict and object formats
                if isinstance(usage, dict):
                    prompt = usage.get("prompt_tokens") or usage.get("input_tokens", 0)
                    completion = usage.get("completion_tokens") or usage.get("output_tokens", 0)
                    total = usage.get("total_tokens", 0)
                else:
                    prompt = getattr(usage, "prompt_tokens", None) or getattr(usage, "input_tokens", 0)
                    completion = getattr(usage, "completion_tokens", None) or getattr(usage, "output_tokens", 0)
                    total = getattr(usage, "total_tokens", 0)
                
                # Add to totals
                self.prompt_tokens += prompt
                self.completion_tokens += completion
                self.total_tokens += total
                
                # Track this individual LLM call with captured context
                call_info = {
                    "call_number": self.llm_call_count,
                    "prompt_tokens": prompt,
                    "completion_tokens": completion,
                    "total_tokens": total,
                    "context": self.llm_context or "unknown",  # Use captured context
                    "timestamp": time.perf_counter() - self.session_start
                }
                self.llm_calls.append(call_info)
                
                print(f"[Tokens Call #{self.llm_call_count} ({self.llm_context})] Prompt: {prompt}, Completion: {completion}, Total: {total}")
                
                # Clear context for next call
                self.llm_context = None
            else:
                print("[Warning] No token usage found in LLM response")
                
        except Exception as e:
            print(f"[Error extracting tokens] {e}")
            import traceback
            traceback.print_exc()

    # -----------------------
    # GENERATION (time from last tool to end)
    # -----------------------
    def on_llm_new_token(self, token, **kwargs):
        """Track when generation starts (first token after last tool)"""
        if self.last_tool_end_time and self.generation_start_time is None:
            self.generation_start_time = time.perf_counter()

    def finalize(self):
        # Calculate generation time as: time from last tool end to session end
        if self.last_tool_end_time:
            self.generate_total = time.perf_counter() - self.last_tool_end_time
        self.session_end = time.perf_counter()

    # -----------------------
    # SUMMARY
    # -----------------------
    def get_summary(self):
        t_tool_total = sum(t for _, t in self.tool_timings)
        t_retrieve = sum(
            t for name, t in self.tool_timings if "retriev" in name.lower()
        )

        return {
            "T_retrieve": round(t_retrieve, 4),
            "T_tools": [
                {"tool": n, "time": round(t, 4)} for n, t in self.tool_timings
            ],
            "T_tools_total": round(t_tool_total, 4),
            "T_reason": round(self.reasoning_total, 4),
            "T_generate": round(self.generate_total, 4),
            "T_total": round(
                (self.session_end or time.perf_counter()) - self.session_start, 4
            ),
            "prompt_tokens": self.prompt_tokens,
            "completion_tokens": self.completion_tokens,
            "total_tokens": self.total_tokens,
            "llm_calls": self.llm_calls,
            "num_llm_calls": self.llm_call_count
        }
    
    def print_token_breakdown(self):
        """Print a detailed breakdown of token usage per LLM call"""
        print("\n=== Token Usage Breakdown ===")
        print(f"{'Call':<6} {'Prompt':<8} {'Completion':<12} {'Total':<8} {'Context'}")
        print("-" * 70)
        for call in self.llm_calls:
            print(f"#{call['call_number']:<5} {call['prompt_tokens']:<8} {call['completion_tokens']:<12} {call['total_tokens']:<8} {call['context']}")
        print("-" * 70)
        print(f"{'TOTAL':<6} {self.prompt_tokens:<8} {self.completion_tokens:<12} {self.total_tokens:<8}")

In [40]:
# from langchain.schema import HumanMessage, SystemMessage

# prof = Profiler(async_mode="disabled")
# prof.start()
# timing_callback = TimingCallback()

# start_total = time.perf_counter()

# # Build FAISS (if not yet built)
# # faiss_store = build_faiss_index(chunked_docs)

# # Load FAISS Vector Store
# faiss_store = FAISS.load_local(
#     "faiss_index",
#     embeddings=embedding_model,
#     allow_dangerous_deserialization=True
# )

# system_prompt=create_system_message()
# agent = create_agent(faiss_store)

# # Run the Agent with a Query
# question = "Show Operating Expenses for the last 3 fiscal years, year-on-year comparison."

# response = agent.invoke({
#     "input": [
#         SystemMessage(content=system_prompt),
#         HumanMessage(content=question)
#     ]
# }, config={"callbacks": [timing_callback]})
# print(response)

# elapsed_total = time.perf_counter() - start_total

# prof.stop()
# print(prof.output_text(unicode=True, color=True))
# prof.last_session.save("02_Agent.pyisession")
# html_output = prof.output_html()
# with open("02_Agent.html", "w", encoding="utf-8") as f:
#     f.write(html_output)
# print(f"Total elapsed time: {elapsed_total}")


In [78]:
import json
import re

def extract_json_and_prose(response_text: str):
    """
    Extracts JSON (fenced or unfenced) and prose explanation.
    Captures the full JSON, even with nested braces.
    """
    # Match fenced or unfenced JSON
    pattern = re.compile(
        r"```(?:json)?\s*(\{[\s\S]*\})\s*```"
        r"(?:\s*\*\*Prose Explanation:\*\*\s*([\s\S]*))?"
        r"|(\{[\s\S]*\})\s*(?:\*\*Prose Explanation:\*\*\s*([\s\S]*))?",
        re.DOTALL
    )

    match = pattern.search(response_text)
    if not match:
        print("No JSON block found.")
        return None, None

    # Select the matched JSON group (fenced or unfenced)
    json_str = (match.group(1) or match.group(3) or "").strip()
    prose = (match.group(2) or match.group(4) or "").strip()

    # Trim trailing junk after the last closing brace
    last_brace = json_str.rfind("}")
    if last_brace != -1:
        json_str = json_str[:last_brace + 1]

    try:
        data = json.loads(json_str)
    except json.JSONDecodeError as e:
        print(f"JSON parse error: {e}")
        print("Captured JSON preview:\n", json_str[:500])
        data = None

    return data, prose

In [42]:
# response['output']

In [43]:
# data, prose = extract_json_and_prose(response['output'])

In [44]:
# data

In [45]:
# prose = prose.encode('utf-8').decode('unicode_escape')
# print(prose)

In [46]:
# print("\n=== Summary ===")
# for name, t in timing_callback.tool_timings:
#     print(f"{name:<20} {t:.3f}s")
# print(f"{'reasoning':<20} {timing_callback.reasoning_time:.3f}s")

In [ ]:
prof = Profiler(async_mode="disabled")
prof.start()

start_total = time.perf_counter()

# Build FAISS (if not yet built)
faiss_store = build_faiss_index(chunked_docs, embedding_model)

prof.stop()
print(prof.output_text(unicode=True, color=True))
prof.last_session.save("02_BuildingFAISS.pyisession")
html_output = prof.output_html()
with open("02_BuildingFAISS.html", "w", encoding="utf-8") as f:
    f.write(html_output)
print(f"Total elapsed time: {elapsed_total}")


  _     ._   __/__   _ _  _  _ _/_   Recorded: 15:28:54  Samples:  3041
 /_//_/// /_\ / //_// / //_'/ //     Duration: 39.999    CPU time: 4.203
/   _/                      v5.1.1

Profile at C:\Users\USER\AppData\Local\Temp\ipykernel_8508\3294406290.py:2

39.999 <module>  ..\..\..\Temp\ipykernel_8508\3294406290.py:1
└─ 39.999 build_faiss_index  ..\..\..\Temp\ipykernel_8508\2499090043.py:3
   └─ 39.826 FAISS.from_documents  langchain_core\vectorstores\base.py:809
         [49 frames hidden]  langchain_community, langchain_openai...
            25.045 _SSLSocket.read  <built-in>
            10.939 _SSLSocket.read  <built-in>


Total elapsed time: 0.3780076000257395


## 5. Benchmark Runner

Run these 3 standardized queries. Produce JSON then prose answers with citations. These are the standardized queries.

*   Gross Margin Trend (or NIM if Bank)
    *   Query: "Report the Gross Margin (or Net Interest Margin, if a bank) over the last 5 quarters, with values."
    *   Expected Output: A quarterly table of Gross Margin % (or NIM % if bank).

*   Operating Expenses (Opex) YoY for 3 Years
    *   Query: "Show Operating Expenses for the last 3 fiscal years, year-on-year comparison."
    *   Expected Output: A 3-year Opex table (absolute numbers and % change).

*   Operating Efficiency Ratio
    *   Query: "Calculate the Operating Efficiency Ratio (Opex ÷ Operating Income) for the last 3 fiscal years, showing the working."
    *   Expected Output: Table with Opex, Operating Income, and calculated ratio for 3 years.


In [ ]:
# Load the questions from JSON file
with open('qa/nvda_ground_truth3.json', 'r') as f:
    test_questions = json.load(f)

# Extract just the queries
queries = [item['query'] for item in test_questions]

all_results = []
# system_prompt = create_system_message()

for i, question in enumerate(queries, 1):
    print(f"\n{'='*60}")
    print(f"Question {i}: {question}")
    print('='*60)
    
    # Create NEW profiler for each question
    prof = Profiler(async_mode="disabled")
    timing_callback = TimingCallback()
    
    start = time.perf_counter()
    prof.start()
    
    prompt = create_system_message(question)

    # Load FAISS Vector Store
    faiss_store = FAISS.load_local(
        "faiss_index",
        embeddings=embedding_model,
        allow_dangerous_deserialization=True
    )
    agent = create_agent(faiss_store)

    response = agent.invoke(prompt, config={"callbacks": [timing_callback]})
    timing_callback.finalize()

    prof.stop()
    elapsed = time.perf_counter() - start
    
    # Save individual profile files
    profile_session_path = f"03_Agent_q{i}.pyisession"
    profile_html_path = f"03_Agent_q{i}.html"

    # Save individual profile
    print(f"\n--- Profile for Question {i} ---")
    print(prof.output_text(unicode=True, color=True))
    prof.last_session.save(profile_session_path)
    
    with open(profile_html_path, "w", encoding="utf-8") as f:
        f.write(prof.output_html())
    
    # Extract results
    data, prose = extract_json_and_prose(response['output'])
    
    # Store result
    result = {
        "question_number": i,
        "question": question,
        "raw_response": f"{response}",
        "raw_output": response['output'],
        "data": data,
        "prose": prose,
        "elapsed_time": elapsed,
        "callback_timing": timing_callback.get_summary(),
        "profile_session_path": profile_session_path,
        "profile_html_path": profile_html_path
    }
    all_results.append(result)

    # Print summary
    print(f"\n--- Profile for Question {i} ---")
    print(prof.output_text(unicode=True, color=True))
    print(f"\nResponse: {response}")
    print(f"Elapsed time: {elapsed:.2f}s")

# Save all results to a single JSON file
results_filename = f"agent_results.json"

with open(results_filename, 'w', encoding='utf-8') as f:
    json.dump(all_results, f, indent=2, ensure_ascii=False)

print(f"\n{'='*60}")
print(f"All results saved to: {results_filename}")
print(f"{'='*60}")

# Print summary statistics
total_time = sum(r['elapsed_time'] for r in all_results)
avg_time = total_time / len(all_results)
print(f"\nTotal questions: {len(all_results)}")
print(f"Total time: {total_time:.2f}s")
print(f"Average time per question: {avg_time:.2f}s")



Question 1: Report the Gross Margin (%) over the last 5 quarters, with values.


> Entering new AgentExecutor chain...
[LLM reasoning segment] 2.481s
[Tokens Call #1 (initial_planning)] Prompt: 556, Completion: 108, Total: 664
To report the Gross Margin (%) over the last 5 quarters, I need to first retrieve the relevant financial data, specifically the revenue and gross profit figures for the last 5 quarters. Then, I will calculate the Gross Margin (%) using the formula: 

\[ \text{Gross Margin (\%)} = \left( \frac{\text{Gross Profit}}{\text{Revenue}} \right) \times 100 \]

Action: retriever
Action Input: "Gross Profit and Revenue for the last 5 quarters"
[Tool start] retriever
[Tool end] retriever (0.437s)

Observation: ===== Document 1 =====
GAAP			Non-GAAP		
	Q4 FY25	Y/Y	Q/Q	Q4 FY25	Y/Y	Q/Q
Revenue	$39,331	+78%	+12%	$39,331	+78%	+12%
Gross Margin	73.0%	-3.0 pts	-1.6 pts	73.5%	-3.2 pts	-1.5 pts
Operating
Income	$24,034	+77%	+10%	$25,516	+73%	+10%
Net Income	$22,091	+80%	+14%	$22,066

## 6. Instrumentation

Log timings: T_ingest, T_retrieve, T_rerank, T_reason, T_generate, T_total. Log tokens, cache hits, tools.

In [ ]:
# Example instrumentation schema
import pandas as pd
logs = pd.DataFrame(columns=['Query','T_ingest','T_retrieve','T_rerank','T_reason','T_generate','T_total','Tokens','CacheHits','Tools'])
logs

## 7. Optimizations

**Required Optimizations**

Each team must implement at least:
*   2 retrieval optimizations (e.g., hybrid BM25+vector, smaller embeddings, dynamic k).
*   1 caching optimization (query cache or ratio cache).
*   1 agentic optimization (plan pruning, parallel sub-queries).
*   1 system optimization (async I/O, batch embedding, memory-mapped vectors).

In [ ]:
# TODO: Implement optimizations

## 8. Results & Plots

Show baseline vs optimized. Include latency plots (p50/p95) and accuracy tables.

In [ ]:
# TODO: Generate plots with matplotlib
